# 任务 2：使用 SageMaker Clarify
	
在本实验中，您将使用 Amazon SageMaker Clarify 来检测训练前数据和训练后模型中的偏差并访问可解释性报告。


## 任务 2.1：环境设置

安装程序包和依赖项。

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sagemaker_datawrangler
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

接着，导入、拆分和上传数据集。

In [ ]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

现在，训练 XGBoost 模型。您将为 SageMaker Clarify ModelConfig 使用这个经过训练的模型。完成此次训练大约需要 3–4 分钟。

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

## 任务 2.2：设置 SageMaker Clarify 任务并访问偏差报告

要使用 [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html)，您首先要从训练任务中创建一个模型。然后，您需要设置必要的配置并在经过训练的模型上运行 SageMaker Clarify 任务。在此任务中，您将完成以下操作：

- 创建模型。
- 启用 SageMaker Clarify。
- 运行偏差报告。
- 访问该报告。

### 任务 2.2.1：创建模型

从训练任务创建一个模型以与 SageMaker Clarify 一起使用。

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### 任务 2.2.2：启用 SageMaker Clarify

现在，要开始进行配置，请启用 SageMaker Clarify。

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

借助 SageMaker Clarify，您可以通过各种指标来检测数据中潜在的训练前偏差和模型中的训练后偏差。有关配置 Clarify 分析的更多信息，请参阅 [metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html)。

配置 **DataConfig** 对象以将数据 I/O 信息传递给 SageMaker Clarify。在此处，您将指定输入数据集的位置、输出的位置、收入 (**label**) 列、标头名称和数据集类型。

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

配置 **ModelConfig** 对象以传递有关经过训练的模型的信息。设置模型名称和临时专用终端节点以避免额外流量进入生产模型（**instance_type** 和 **instance_count**）。此外，您还需要设置 **accept_type** 来表示终端节点响应负载格式，并设置 **content_type** 来表示向终端节点发送的请求的负载格式。

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

配置 **ModelPredictedLabelConfig** 的 **probability_threshold**，将样本概率转换为二进制标签以进行偏差分析。高于阈值的预测被解释为标签值 **1**，而低于或等于阈值的预测被解释为标签值 **0**。

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

配置 **BiasConfig** 以表示敏感列 (**facets**)、潜在敏感特征 (**facet_values_or_threshold**) 以及有利结果 (**label_values_or_threshold**)。

您可以表示 **facet_values_or_threshold** 和 **label_values_or_threshold** 的分类数据和连续数据。**sex** 和 **age** 特征都属于分类数据。

该模型的目标是确定一个人的收入是否为 50,000 USD 或更多，并且积极的结果是有利的。使用 **BiasConfig** 来提供与以下内容相关的信息：哪些列包含具有 **Sex** 敏感组的分面、哪些敏感特征可能使用 **facet_values_or_threshold** 以及哪些理想结果使用 **label_values_or_threshold**。按 **age** 进行分组以查看是否会因个人年龄而出现差异。

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### 任务 2.2.3：运行偏差报告

使用您的配置创建偏差报告以进行训练前和训练后分析。完成本步骤大约需要 15–20 分钟。在系统生成偏差报告时，您可以参阅 [机器学习在金融业的公平措施](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf) 来了解 SageMaker Clarify 的计算方法。

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### 任务 2.2.4：访问偏差报告

在 SageMaker Studio 中，您可以在 <b>Experiments</b>（实验）选项卡下查看结果。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_8.ipynb** 选项卡拖到一边，或者选择（右键单击）**lab_8.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览偏差报告时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完偏差报告后，通过选择 **lab_8.ipynb** 选项卡返回至笔记本。

1.在 SageMaker Studio 中，选择 **SageMaker 主页**图标。

2.选择 **Experiments**（实验）。

此时 SageMaker Studio 将打开 **Experiments**（实验）选项卡。

3.在 **Experiments**（实验）选项卡中，选择左侧的 **Unassigned runs**（未分配的运行）。

4.在 **Unassigned runs**（未分配的运行）列表中，选择标题中包含 **clarify-bias-** 的训练任务名称。

5.在 **Experiments**（实验）选项卡中，选择左侧的 **Bias Reports**（偏差报告）。

在 SageMaker Clarify 运行完偏差任务后，系统将提供一份包含多个指标的偏差报告。

6.等待 SageMaker Clarify 运行完偏差任务。

每个偏差指标都有详细的解释，并含有示例，您可以通过选择每个值来加以探索。

7.通过选择偏差指标旁的箭头并展开字段来检查 **Class Imbalance**（类不平衡）和 **Disparate (Adverse) Impact (DI)**（分散（不利）影响 (DI)）详细解释。

浏览完偏差指标后，继续执行下一个任务。

## 任务 2.3：访问可解释性报告

除了使用偏差报告之外，您还可以使用 SageMaker Clarify 来分析单个预测的本地解释。创建一份报告以查看 SageMaker Clarify 生成的预测的可解释性结果并分析报告中的关键指标。在此任务中，您将完成以下操作：

1.定义夏普里值 (SHAP) 配置。
2.运行可解释性报告。
3.访问可解释性报告。

### 任务 2.3.1：定义 SHAP 配置

您可以运行可解释性报告，以获得对一个模型为什么做出特定预测的解释。SageMaker Clarify 使用 SHAP 来生成关于模型决策推理的报告。有关 SHAP 基准的更多信息，请参阅 [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)。

各个 SHAP 指标的配置如下：
- **baseline**：行列表或 Amazon Simple Storage Service (Amazon S3) 对象 URI，用作 Kernel SHAP 算法中的基准数据集。
- **num_samples**：Kernel SHAP 算法中所用的样本数量。此数值决定了所生成的用于计算 SHAP 值的合成数据集的大小。
- **agg_method: mean_abs**：所有实例的绝对 SHAP 值的平均值。
- **save_local_shap_values**：指示是否会将本地 SHAP 值保存在输出位置的布尔值。

有关所用指标的更多信息，请参阅 [SHAP metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html)。

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### 任务 2.3.2：运行可解释性报告

使用您的配置创建可解释性报告。完成本步骤大约需要 10–15 分钟。在系统生成可解释性报告时，您可以通过继续执行下一个任务，在 SageMaker Studio 中监控任务状态。

有关 SageMaker Clarify 可解释性流程的更多信息，请参阅 [Amazon AI Fairness and Explainability Whitepaper](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)。

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### 任务 2.3.3：访问可解释性报告

与偏差报告一样，您可以在 Studio 中的 <b>Experiments</b>（实验）选项卡下查看可解释性报告。

1.选择 **Experiments**（实验）选项卡。

2.在 **Experiments**（实验）选项卡中，选择左侧的 **Unassigned runs**（未分配的运行）。

3.在 **Unassigned runs**（未分配的运行）列表中，选择标题中包含 **clarify-bias-** 的训练任务名称。

4.在 **Experiments**（实验）选项卡中，选择左侧的 **Explainability**（可解释性）。

5.完成可解释性任务后，要查看 **Feature Importance**（特征重要性）图表，请选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">View sample notebook</span>（查看示例笔记本）。

此时 SageMaker Studio 将在选项卡中打开 **Fairness and Explainability with SageMaker Clarify**（利用 SageMaker Clarify 实现公平性和可解释性）笔记本。

哪些特征是最重要的？哪些特征是最不重要的？ 是否有任何出乎意料的结果？

### 总结

恭喜！ 您已经使用 SageMaker Clarify 创建了可用于开发更可靠且更具合规性的模型的偏差和可解释性报告。在下一个实验中，您将部署模型并运行推理。您将在下一个实验中继续使用您的模型。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。